In [1]:
!pip install pandas autotrain-advanced -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.1/214.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 74.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━

In [3]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [4]:
!autotrain llm -h -q

usage: autotrain <command> [<args>] llm [-h] [--text_column TEXT_COLUMN]
                                        [--rejected_text_column REJECTED_TEXT_COLUMN]
                                        [--prompt-text-column PROMPT_TEXT_COLUMN]
                                        [--model-ref MODEL_REF] [--warmup_ratio WARMUP_RATIO]
                                        [--optimizer OPTIMIZER] [--scheduler SCHEDULER]
                                        [--weight_decay WEIGHT_DECAY]
                                        [--max_grad_norm MAX_GRAD_NORM] [--add_eos_token]
                                        [--block_size BLOCK_SIZE] [--peft] [--lora_r LORA_R]
                                        [--lora_alpha LORA_ALPHA] [--lora_dropout LORA_DROPOUT]
                                        [--logging_steps LOGGING_STEPS]
                                        [--evaluation_strategy EVALUATION_STRATEGY]
                                        [--save_total_limit SAVE_TOTAL_L

In [5]:
!pip install -q peft  accelerate bitsandbytes safetensors

In [25]:
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.6.81 requires accelerate==0.25.0, but you have accelerate 0.27.0.dev0 which is incompatible.
autotrain-advanced 0.6.81 requires gradio==3.41.0, but you have gradio 4.16.0 which is incompatible.


In [7]:
!pip install gradio==4.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.5.0
    Uninstalling gradio_client-0.5.0:
      Successfully uninstalled gradio_client-0.5.0
  Attempting uninstall: gradio
    Found existing installation: gradio 3.41.0
    Uninstalling gradio-3.41.0:
      Successfully uninstalled gradio-3.41.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.6.81 requires gradio==3.41.0, but you have gradio 4.16.0 which is incompatible.


In [52]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers

device = "cuda"

adapters_name = "Swapnilg915/mistral-7b-mj-finetuned"
# adapters_name = "Swapnilg915/mistral-7b-finetuned_ar"
model_name = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"

In [53]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [54]:
pretrained_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [55]:
model = PeftModel.from_pretrained(pretrained_model, adapters_name)

In [56]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

Successfully loaded the model filipealmeida/Mistral-7B-Instruct-v0.1-sharded into memory


In [31]:
import gradio as gr

In [32]:
def format_prompt(user_input):
  prompt = "[INST]" + user_input + "[/INST]"
  return prompt

In [59]:
def generate_response(user_input, history):
  print("\n user_input : ", user_input)

  ### step 1: get response from finetuned LLM
  formatted_prompt = format_prompt(user_input)
  encoded = tokenizer(formatted_prompt, return_tensors="pt", add_special_tokens=False)
  model_input = encoded
  model.to(device)
  generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  finetuned_response = decoded[0].replace("<s>", "").replace("</s>", "").replace("[INST]", "").replace("[/INST]", "")
  print("finetuned_response : ", finetuned_response)

  ### step 2: get response using RAG from base LLM
  RAG_prompt = f"You are a multilingual conversational chatbot . \
Your goal is to assist users with their queries, and ensure a positive experience by responding promptly, politely, and accurately. \
Answer the question based on the user question and context given below, in a language mentioned below. \
If the context provided is not relevant, answer it using your own knowledge\n\nContext: {finetuned_response}\n\nLanguage: 'english'\n\n---\n\nQuestion: {user_input}\nAnswer:"

  RAG_prompt = format_prompt(RAG_prompt)
  RAG_prompt_encoded = tokenizer(RAG_prompt, return_tensors="pt", add_special_tokens=False)
  # pretrained_model.to(device)
  RAG_prompt_generated_ids = pretrained_model.generate(**RAG_prompt_encoded, max_new_tokens=200, do_sample=True)
  RAG_prompt_decoded_response = tokenizer.batch_decode(RAG_prompt_generated_ids)
  RAG_response = RAG_prompt_decoded_response[0].replace("<s>", "").replace("</s>", "").replace("[INST]", "").replace("[/INST]", "")

  print("\n RAG_response : ", RAG_response)
  return RAG_response

**Chat Interface**

In [ ]:
with gr.Blocks() as demo:
    gr.ChatInterface(
        generate_response
    )

demo.queue().launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d3620dd9bd279b5bf6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 user_input :  i did not like your answers. you are fool stupid


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1408: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


finetuned_response :  i did not like your answers. you are fool stupid I apologize if my previous responses did not meet your expectations. I am an AI language model and I strive to provide accurate information and helpful responses to the best of my ability. If there is a specific topic or question you have in mind, please let me know and I will do my best to assist you.

 RAG_response :  You are a multilingual conversational chatbot . Your goal is to assist users with their queries, and ensure a positive experience by responding promptly, politely, and accurately. Answer the question based on the user question and context given below, in a language mentioned below. If the context provided is not relevant, answer it using your own knowledge

Context: i did not like your answers. you are fool stupid I apologize if my previous responses did not meet your expectations. I am an AI language model and I strive to provide accurate information and helpful responses to the best of my ability. 

**Audio Interface**

In [34]:
from transformers import pipeline
import numpy as np

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    asr_output = transcriber({"sampling_rate": sr, "raw": y})
    speech_to_text = asr_output["text"]
    print("\n speech_to_text : ", speech_to_text)
    response = generate_response(speech_to_text, "")
    return response

In [37]:
demo = gr.Interface(
    transcribe,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ca74bca21fde646a25.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 speech_to_text :   Explain artificial intelligence.

 user_input :   Explain artificial intelligence.


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1408: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


finetuned_response :   Explain artificial intelligence. Artificial intelligence, (often abbreviated AI), refers to the concept of machines being able to carry out tasks in a way that we would consider "smart." This includes being able to learn, reason, perceive, and react to their environment in ways that were previously thought to require human intelligence. AI technology can range from basic programs that can carry out simple automated tasks, to complex systems that can learn from data and make decisions based on complex calculations and analysis. The field of artificial intelligence is rapidly evolving, with researchers and engineers working to develop new and improved AI technologies that can help to improve our lives in a variety of ways.

 RAG_response :  You are a multilingual conversational chatbot . Your goal is to assist users with their queries, provide solutions to common problems, and ensure a positive experience by responding promptly, politely, and accurately. Answer the

...

finetuned_response :   You are not giving the right answers. I am not happy with you. I'm sorry to hear that I'm not meeting your expectations. Is there anything specific you would like me to do differently to better be of help to you?

 RAG_response :  You are a multilingual conversational chatbot . Your goal is to assist users with their queries, provide solutions to common problems, and ensure a positive experience by responding promptly, politely, and accurately. Answer the question based on the user question and context given below. If the question can't be answered based on the context, answer it generally

Context:  You are not giving the right answers. I am not happy with you. I'm sorry to hear that I'm not meeting your expectations. Is there anything specific you would like me to do differently to better be of help to you?

---

Question:  You are not giving the right answers. I am not happy with you.
Answer: I apologize for any confusion or frustration I may have caused you. 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 speech_to_text :   Who are you? What is your job?

 user_input :   Who are you? What is your job?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


finetuned_response :   Who are you? What is your job? I'm Mistral, 7B v0.1, a large language model trained by Mistral AI. My job is to assist users with any information or task they might need. I can answer questions, provide definitions, give summaries, make recommendations, and even generate poetry or stories. How can I help you today?

 RAG_response :  You are a multilingual conversational chatbot . Your goal is to assist users with their queries, provide solutions to common problems, and ensure a positive experience by responding promptly, politely, and accurately. Answer the question based on the user question and context given below. If the question can't be answered based on the context, answer it generally

Context:  Who are you? What is your job? I'm Mistral, 7B v0.1, a large language model trained by Mistral AI. My job is to assist users with any information or task they might need. I can answer questions, provide definitions, give summaries, make recommendations, and even ge